In [ ]:
import os
os.chdir('/home/yash/Desktop/Decision-Flip-Experiments')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn import ModeKeys, Estimator
import _pickle as pickle
from scipy.misc import imread
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from fgsm_cifar import fgsm
from fgsm_cifar_wrt_class import fgsm_wrt_class
import sys  
import matplotlib.patches as mpatches

img_rows = 32
img_cols = 32
img_chas = 3
input_shape = (img_rows, img_cols, img_chas)
n_classes = 10

def save_obj(obj, name ):
    with open('data/misc/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name ):
    with open('data/misc/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

sd = 'shape_dict'

def save_as_txt(label,ar):
    d=load_obj(sd)
    d[label] = ar.shape
    save_obj(d,sd)
    X = ar.reshape((ar.shape[0],-1))
    np.savetxt(label , X)

def load_from_txt(label):
    d=load_obj(sd)
    X= np.loadtxt(label)
    return X.reshape(d[label])

def load_CIFAR10(ROOT):
  """ load all of cifar """
  xs = []
  ys = []
  for b in range(1,6):
    f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
    X, Y = load_CIFAR_batch(f)
    xs.append(X)
    ys.append(Y)    
  Xtr = np.concatenate(xs)
  Ytr = np.concatenate(ys)
  del X, Y
  Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
  return Xtr, Ytr, Xte, Yte

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = pickle.load(f,encoding='latin1')
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
    Y = np.array(Y)
    return X, Y

def make_label(i,m,e,n,r):
    if(r == False):
        return i + "_m" + str(m) + "_e" + str(e) + "_n" + str(n)
    else:
        base = i + "_m" + str(m) + "_e" + str(e) + "_n" + str(n) + "_r"
        lrn = base + "normal"
        return base, lrn

def get_flip_path(l):
    return 'data/cifar/' + l+ '.txt'

def get_misc_path(l):
    return 'data/misc/' + l+ '.txt'

def random_normal_func(X, n, save, lr, lrn):
    X=X.reshape(-1,img_rows*img_cols*img_chas)
    mean, std = np.mean(X, axis=0), np.std(X,axis=0)
    randomX = np.zeros([n,X[0].size])
    for i in range(X[0].size):
        randomX[:,i] = np.random.normal(mean[i],std[i],n)
    randomX = randomX.reshape(-1,img_rows,img_cols,img_chas)
    X_random_normal = randomX
    ans = sess.run(env.ybar, feed_dict={env.x: randomX,env.training: False})
    y_random_normal = _to_categorical(np.argmax(ans,axis=1), n_classes)
    X_random = np.random.rand(n,img_rows,img_cols,img_chas)
    y_random = sess.run(env.ybar, feed_dict={env.x: X_random,env.training: False})
    y_random = _to_categorical(np.argmax(y_random, axis=1), n_classes)
    
    if(save):
        save_as_txt(get_flip_path(lr),X_random)
        save_as_txt(get_flip_path(lrn),X_random_normal)
    
    return X_random, y_random, X_random_normal, y_random_normal  

def make_data():
    y_train_sub = _predict(X_train,env)
    y_train_sub = _to_categorical(np.argmax(y_train, axis=1), n_classes)
    y_test_sub = _predict(X_test,env)
    y_test_sub = _to_categorical(np.argmax(y_test, axis=1), n_classes)
    
    return X_test, y_test_sub, X_train, y_train_sub

def run_flip(method, epochs):
    save_obj({},sd)
    #n is len here!
    X_test, y_test, X_train, y_train = make_data()
    
    test_label = make_label("test", method, epochs, len(X_train), False)
    train_label = make_label("train", method, epochs,len(X_test), False)
    
    X_flip_per_class_test = create_flip_wrt_class(X_test, y_test, test_label)
    X_flip_per_class_train = create_flip_wrt_class(X_train, y_train, train_label)
    

def restore_flip(method, epochs):
    test_label = make_label("test", method, epochs,len(X_train), False)
    train_label = make_label("train", method, epochs,len(X_train), False)

    X_flip_per_class_test = load_from_txt(get_flip_path(test_label))
    X_flip_per_class_train = load_from_txt(get_flip_path(train_label))

def _to_categorical(x, n_classes):
    x = np.array(x, dtype=int).ravel()
    n = x.shape[0]
    ret = np.zeros((n, n_classes))
    ret[np.arange(n), x] = 1
    return ret

In [ ]:
print('\nLoading CIFAR10')
ab=sys.getdefaultencoding()
print(ab)
cifar10_dir = 'cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

X_train = X_train.reshape(-1, img_rows, img_cols, img_chas)
X_test = X_test.reshape(-1, img_rows, img_cols, img_chas)

y_train = _to_categorical(y_train, n_classes)
y_test = _to_categorical(y_test, n_classes)

In [ ]:
print('\nLoading pre-Shuffled training data')
ind = np.loadtxt(get_misc_path('ind'), dtype = int)
X_train, y_train = X_train[ind], y_train[ind]

class Dummy:
    pass
env = Dummy()

In [ ]:
def model(x, logits=False, training=False):
    conv0 = tf.layers.conv2d(x, filters=32, kernel_size=[3, 3],
                             padding='same', name='conv0',
                             activation=tf.nn.relu)
    
    pool0 = tf.layers.max_pooling2d(conv0, pool_size=[2, 2],
                                    strides=2, name='pool0')
    
    conv1 = tf.layers.conv2d(pool0, filters=64,
                             kernel_size=[3, 3], padding='same',
                             name='conv1', activation=tf.nn.relu)
   
 
    
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2],
                                    strides=2, name='pool1')
    
    conv2 = tf.layers.conv2d(pool1, filters=128,
                             kernel_size=[1,1], padding='same',
                             name='conv2', activation=tf.nn.relu)
    

    
    flat = tf.reshape(conv2, [-1, 8*8*128], name='flatten')
    
    dense1 = tf.layers.dense(flat, units= 1024, activation=tf.nn.relu,
                            name='dense1')
    
    dense2 = tf.layers.dense(dense1, units=128, activation=tf.nn.relu,
                            name='dense2')
    logits_ = tf.layers.dense(dense2, units=10, name='logits') #removed dropout
    
    y = tf.nn.softmax(logits_, name='ybar')
    
    if logits:
        return y, logits_
    return y

In [ ]:
# We need a scope since the inference graph will be reused later
with tf.variable_scope('model'):
    env.x = tf.placeholder(tf.float32, (None, img_rows, img_cols,
                                        img_chas), name='x')
    env.y = tf.placeholder(tf.float32, (None, n_classes), name='y')
    env.training = tf.placeholder(bool, (), name='mode')

    env.ybar, logits = model(env.x, logits=True,
                             training=env.training)

    z = tf.argmax(env.y, axis=1)
    zbar = tf.argmax(env.ybar, axis=1)
    env.count = tf.cast(tf.equal(z, zbar), tf.float32)
    env.acc = tf.reduce_mean(env.count, name='acc')

    xent = tf.nn.softmax_cross_entropy_with_logits(labels=env.y,
                                                   logits=logits)
    env.loss = tf.reduce_mean(xent, name='loss')

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(extra_update_ops):
    env.optim = tf.train.AdamOptimizer(beta1=0.9, beta2=0.999, epsilon=1e-08,).minimize(env.loss) 


In [ ]:
with tf.variable_scope('model', reuse=True):
    for i in range(n_classes):
        if(i==0):
            env.x_adv_wrt_class = (fgsm_wrt_class(model, env.x, i, step_size=.5, bbox_semi_side=10))
        else:
            x = (fgsm_wrt_class(model, env.x, i, step_size=.05, bbox_semi_side=10))
            env.x_adv_wrt_class = tf.concat([env.x_adv_wrt_class, x],axis=0)
    env.x_adv, env.all_flipped = fgsm(model, env.x, step_size=.05, bbox_semi_side=10) #epochs is redundant now!
    

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [ ]:
def save_model(label):
    saver = tf.train.Saver()
    saver.save(sess,  './models/cifar/' + label)
    
def restore_model(label):
    saver = tf.train.Saver()
    saver.restore(sess, './models/cifar/' + label)

In [ ]:
def _evaluate(X_data, y_data, env):
    print('\nEvaluating')
    n_sample = X_data.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    loss, acc = 0, 0
    ns = 0
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        batch_loss, batch_count, batch_acc = sess.run(
            [env.loss, env.count, env.acc],
            feed_dict={env.x: X_data[start:end],
                       env.y: y_data[start:end],
                       env.training: False})
        loss += batch_loss*batch_size
        ns+=batch_size
        acc += batch_acc*batch_size
    loss /= ns
    acc /= ns
    print(' loss: {0:.4f} acc: {1:.4f}'.format(loss, acc))
    return loss, acc

In [ ]:
def _predict(X_data, env):
    print('\nPredicting')
    n_sample = X_data.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    yval = np.empty((X_data.shape[0], n_classes))
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        batch_y = sess.run(env.ybar, feed_dict={
            env.x: X_data[start:end], env.training: False})
        yval[start:end] = batch_y
    return yval

def train(label):
    print('\nTraining')
    n_sample = X_train.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    n_epoch = 50
    for epoch in range(n_epoch):
        print('Epoch {0}/{1}'.format(epoch+1, n_epoch))
        for ind in range(n_batch):
            print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
            start = ind*batch_size
            end = min(n_sample, start+batch_size)
            sess.run(env.optim, feed_dict={env.x: X_train[start:end],
                                           env.y: y_train[start:end],
                                           env.training: True})
        if(epoch%5 == 0):
            model_label = label+ '{0}'.format(epoch)
            print("saving model " + model_label)
            save_model(model_label)
            
    save_model(label)

def train_again(X, y, epochs):
    #Not making batches, do that if size > 128
    for i in range(X.shape[0]):
        for e in range(epochs):
            sess.run(env.optim, feed_dict={env.x: [X[i]],
                                           env.y: [y[i]],
                                           env.training: True})
            
    
    

In [ ]:
def create_flip_wrt_class(X, Y, label):
    print('\nCrafting Flipped')
    n_sample = X.shape[0]
    pred = np.argmax(Y,axis=1)
    batch_size = 1
    n_batch = int(np.ceil(n_sample/batch_size))
    x_adv_shape = list(X.shape)[1:]
    x_adv_shape = np.append(np.append(n_sample,n_classes),x_adv_shape)
    X_adv = np.empty(x_adv_shape)
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        tmp = sess.run(env.x_adv_wrt_class, feed_dict={env.x: X[start:end],
                                             env.y: Y[start:end],
                                             env.training: False})
        tmp[pred[start]] = X[start]
        X_adv[start:end] = tmp
    print('\nSaving Flipped')
    os.makedirs('data', exist_ok=True)
    save_as_txt(get_flip_path(label), X_adv)
    return X_adv

In [ ]:
method = 2
epochs = 200
label="cifar_with_cnn"
cls = -1

X_test = X_test[2100:2120]
y_test = y_test[2100:2120]

restore_model(label + str(epochs))
run_flip(method,epochs)

